In [2]:
import pandas as pd
import numpy as np

import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras import optimizers

np.random.seed(7)
from sklearn.preprocessing import Normalizer
from keras.callbacks import EarlyStopping

from keras import backend as K
from keras.utils.generic_utils import get_custom_objects
import numpy as np


Using TensorFlow backend.


In [3]:
headerInput = ['Hora', 'Temperature', 'Wind_Speed', 'Wind_Direction', 'Humidity', 'Air_Density', 'Jardim_PMV','Jardim_Temp','Sala_PMV','Sala_Temp','Mezanino_PMV','Mezanino_Temp']
headerOutput = ['JanelaSalaNorte','PortaSalaLeste','JanelaSalaSul','JanelaSalaOeste','PortaCozinhaLeste','JanelaCozinhaSul','JanelaMezaninoNorte','JanelaMezaninoSul','JanelaMezaninoOeste']

print len(headerInput)
print len(headerOutput)

12
9


In [4]:
Input = pd.read_csv("Selected/v02_input.csv")
Output = pd.read_csv("Selected/v02_output.csv")

Input_pd = Input[headerInput]
Output_pd = Output[headerOutput]

scaler_in = Normalizer().fit(Input_pd)
Input_Norm = scaler_in.transform(Input_pd)
Input_np = np.array(Input_pd)

Output_Norm = np.array(Output_pd)


**Cria** entradas para testes na rede <br>
**Normaliza** entradas para testes na rede 

In [5]:
header = ['Mes', 'Dia', 'Hora', 
         'Temperature', 'Wind_Speed', 'Wind_Direction', 'Humidity', 'Air_Density', 'Rain_Status',
         'AreaServico_PMV', 'AreaServico_Temp', 'Circulacao_PMV', 'Circulacao_Temp', 'Casal_PMV', 'Casal_Temp', 'Banheiro_PMV', 'Banheiro_Temp', 'Jardim_PMV', 'Jardim_Temp', 'Solteiro_PMV', 'Solteiro_Temp', 'Sala_PMV', 'Sala_Temp', 'Mezanino_PMV', 'Mezanino_Temp'
         ]
verao = pd.DataFrame([[11, 12, 13.000000, 32.800000, 3.500000, 322.000000, 39.000000, 1.066291, 0.000000, 0.489916, 27.627043, 0.442085, 27.478200, -0.514111, 29.523684, 0.423624, 27.716838, 2.027013, 32.011196, -1.767618, 25.996711, 0.060116, 25.994876, 1.637796, 31.163378]],
                     columns=header)
inverno = pd.DataFrame([[7.000000, 12.000000, 4.000000, 1.500000, 4.900000, 299.000000, 99.000000, 1.214668, 0.000000, -1.898431, 16.127440, -1.565105, 17.518529, -3.657721, 19.481752, -2.524125, 13.445211, -2.425717, 13.586386,-3.270185, 20.709248, -1.873544, 19.015872, -1.613798, 18.001001]],
                      columns=header)

verao = verao[headerInput]
inverno = inverno[headerInput]

verao_norm = scaler_in.transform(verao)
inverno_norm = scaler_in.transform(inverno)

In [6]:
model = Sequential()
model.add(Dense(40, input_shape=(12, ), kernel_initializer='random_normal', activation='tanh'))
model.add(Dense(40, activation='tanh'))
model.add(Dense(40, activation='tanh'))
model.add(Dense(40, activation='tanh'))
model.add(Dense(40, activation='relu'))
model.add(Dense(9, activation='sigmoid'))
model.summary()
sgd = optimizers.SGD(lr=0.1, decay=1e-5, momentum=0.3, nesterov=True)


model.compile(optimizer=sgd, loss='mean_squared_error', metrics=['mae', 'acc'])

model.fit(Input_np, Output_Norm, validation_split=0.3, epochs=35, batch_size=100)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 40)                520       
_________________________________________________________________
dense_2 (Dense)              (None, 40)                1640      
_________________________________________________________________
dense_3 (Dense)              (None, 40)                1640      
_________________________________________________________________
dense_4 (Dense)              (None, 40)                1640      
_________________________________________________________________
dense_5 (Dense)              (None, 40)                1640      
_________________________________________________________________
dense_6 (Dense)              (None, 9)                 369       
Total params: 7,449
Trainable params: 7,449
Non-trainable params: 0
_________________________________________________________________
Train on

In [7]:
teste_verao = model.predict(verao_norm)
teste_inverno = model.predict(inverno_norm)
print "Verão"
print teste_verao
print "Inverno"
print teste_inverno

Verão
[[ 0.49507231  0.48104483  0.49271482  0.48310167  0.49004775  0.48541814
   0.50177163  0.47992846  0.48745358]]
Inverno
[[ 0.48328418  0.47243547  0.47534117  0.4718793   0.47618115  0.46995202
   0.49132237  0.46514803  0.46658549]]


In [8]:
Output_pd.head()

,JanelaSalaNorte,PortaSalaLeste,JanelaSalaSul,JanelaSalaOeste,PortaCozinhaLeste,JanelaCozinhaSul,JanelaMezaninoNorte,JanelaMezaninoSul,JanelaMezaninoOeste
0,0.30,0.30,0.20,0.67,0.72,0.01,0.50,0.74,0.20
1,0.75,0.78,0.77,0.23,0.78,0.21,0.29,0.70,0.48
2,0.27,0.23,0.66,0.23,0.49,0.31,0.48,0.89,0.51
3,0.62,0.43,0.66,0.90,0.50,0.99,0.99,0.31,0.21
4,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
